In [25]:
import numpy as np
import pickle
from solution import SVMEx
import pandas as pd

In [2]:
# Load t h e p i c k l e f i l e
with open ( 'train_data.pkl' , 'rb' ) as f :
    data = pickle.load ( f )
# Access images and l a b e l s
images = data ['images']
labels = data ['labels']

In [3]:
images=np.array(images)
labels=np.array(labels)

In [4]:
images.shape

(97477, 28, 28)

In [5]:
labels.shape

(97477,)

In [6]:
flattened_images = images.reshape(images.shape[0], -1)
print(flattened_images.shape)

(97477, 784)


In [27]:
np.random.seed(42)
num_samples = flattened_images.shape[0]

indices = np.arange(num_samples)
np.random.shuffle(indices)
shuffled_images = flattened_images[indices]
shuffled_labels = labels[indices]

baseline1Idxs = int(num_samples * 0.8)  # 80% for training, 20% for validation
split_index=int(baseline1Idxs*0.8)

train_images = shuffled_images[:split_index]
train_labels = shuffled_labels[:split_index]

val_images = shuffled_images[split_index:]
val_labels = shuffled_labels[split_index:]

In [28]:
svm=SVMEx()
weights_c1,history_c1=svm.fit(X_train=train_images, y_train=train_labels, 
                                X_val=val_images, y_val=val_labels,
                        num_classes=4, epochs=100,
                        learning_rate=0.001, C=0, batch_size=32)

Epoch 1: Train Loss = 10.0227, Val Loss = 10.0306, Train Acc = 0.5249, Val Acc = 0.5225
Epoch 2: Train Loss = 9.7994, Val Loss = 9.8158, Train Acc = 0.5426, Val Acc = 0.5399
Epoch 3: Train Loss = 9.6511, Val Loss = 9.6759, Train Acc = 0.5684, Val Acc = 0.5662
Epoch 4: Train Loss = 9.5111, Val Loss = 9.5362, Train Acc = 0.5751, Val Acc = 0.5724
Epoch 5: Train Loss = 9.4254, Val Loss = 9.4546, Train Acc = 0.5770, Val Acc = 0.5735
Epoch 6: Train Loss = 9.3221, Val Loss = 9.3535, Train Acc = 0.5931, Val Acc = 0.5878
Epoch 7: Train Loss = 9.2540, Val Loss = 9.2866, Train Acc = 0.5983, Val Acc = 0.5933
Epoch 8: Train Loss = 9.1933, Val Loss = 9.2246, Train Acc = 0.6007, Val Acc = 0.5967
Epoch 9: Train Loss = 9.1384, Val Loss = 9.1728, Train Acc = 0.6087, Val Acc = 0.6048
Epoch 10: Train Loss = 9.0978, Val Loss = 9.1306, Train Acc = 0.6091, Val Acc = 0.6059
Epoch 11: Train Loss = 9.0678, Val Loss = 9.1019, Train Acc = 0.6083, Val Acc = 0.6047
Epoch 12: Train Loss = 9.0291, Val Loss = 9.0670, 

In [29]:
with open ( 'test_data.pkl' , 'rb' ) as f :
    data = pickle.load ( f )
# Access images and l a b e l s
testimages = data ['images']


In [30]:
testimages=np.array(images)

In [31]:
flattened_test_images = testimages.reshape(testimages.shape[0], -1)

In [32]:
test_preds=svm.infer(flattened_test_images,weights_c1)

In [34]:
test_preds==2

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
def make_original_array(preds):
        """
       
        """

        index_column = np.arange(1, preds.size + 1)

        # Reshape index_column and labels_train_modified to 2D column vectors
        index_column = index_column.reshape(-1, 1)
        labels_column = preds.reshape(-1, 1)

        # Concatenate the index column with the labels column
        combined_array = np.hstack((index_column, labels_column))

        return combined_array

In [24]:
predictions=make_original_array(test_preds)

In [26]:
df = pd.DataFrame(predictions, columns=['ID', 'label'])

# Save the DataFrame to a CSV file
df.to_csv('SVM_baseline1_attempt.csv', index=False)

## Random Forest attempt
Parameters: 100 estimators, random_state=42

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf.fit(train_images, train_labels)

# Predict on validation set
val_preds = rf.predict(val_images)

# Calculate accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
print(f'Validation Accuracy: {val_accuracy}')

# Predict on test set
test_preds_rf = rf.predict(flattened_test_images)

# Convert predictions to original array format
predictions_rf = make_original_array(test_preds_rf)

# Create a DataFrame and save to CSV
df_rf = pd.DataFrame(predictions_rf, columns=['ID', 'label'])
df_rf.to_csv('RandomForest_baseline1_attempt.csv', index=False)

## Convolutional neural network attempt
Preprocessing: Reshape images to 4D tensors, reshape labels to categorical variables
CNN: 1st conv2d 32 channels, kernel size 3x3, relu, maxpooling 2x2, 2nd one 64 channels, kernel size 3x3, relu, maxpooling 2x2, flatten, to feed into 2 dense layers: 1st 128, relu, 2nd one 4, softmax for output

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import torch
from torch.utils.data import DataLoader, TensorDataset

# Reshape images to 4D tensor for CNN input
train_images_cnn = train_images.reshape(-1, 28, 28, 1)  # Assuming images are 28x28 pixels
val_images_cnn = val_images.reshape(-1, 28, 28, 1)
flattened_test_images_cnn = flattened_test_images.reshape(-1, 28, 28, 1)

# Convert labels to categorical
train_labels_cnn = to_categorical(train_labels, num_classes=4)
val_labels_cnn = to_categorical(val_labels, num_classes=4)

# Build the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(train_images_cnn, train_labels_cnn, epochs=10, batch_size=32, validation_data=(val_images_cnn, val_labels_cnn))

# Predict on test set
test_preds_cnn = cnn_model.predict(flattened_test_images_cnn)
test_preds_cnn_labels = np.argmax(test_preds_cnn, axis=1)

# Convert predictions to original array format
predictions_cnn = make_original_array(test_preds_cnn_labels)

# Create a DataFrame and save to CSV
df_cnn = pd.DataFrame(predictions_cnn, columns=['ID', 'label'])
df_cnn.to_csv('CNN_baseline1_attempt.csv', index=False)



**Same model using Pytorch**

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convert data to PyTorch tensors
train_images_tensor = torch.tensor(train_images_cnn, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
val_images_tensor = torch.tensor(val_images_cnn, dtype=torch.float32).to(device)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long).to(device)
test_images_tensor = torch.tensor(flattened_test_images_cnn, dtype=torch.float32).to(device)

# Create DataLoader
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, activation='relu')
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, activation='relu')
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

# Initialize the model, loss function, and optimizer
cnn_model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    cnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validate the model
    cnn_model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = cnn_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = val_correct / len(val_dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss}, Accuracy: {val_accuracy}')

# Predict on test set
cnn_model.eval()
with torch.no_grad():
    test_outputs = cnn_model(test_images_tensor)
    _, test_preds_cnn_labels = torch.max(test_outputs, 1)

# Convert predictions to original array format
predictions_cnn = make_original_array(test_preds_cnn_labels.numpy())

# Create a DataFrame and save to CSV
df_cnn = pd.DataFrame(predictions_cnn, columns=['ID', 'label'])
df_cnn.to_csv('CNN_baseline1_attempt.csv', index=False)

# This cell defines and compiles a ResNet50 model using TensorFlow and Keras.
# The model is designed for image classification tasks and consists of the following components:
# 
# 1. **Base Model**: 
#    - Architecture: ResNet50
#    - Pre-trained Weights: ImageNet
#    - Include Top: False (excluding the top fully connected layer)
#    - Input Shape: (28, 28, 3) (assuming input images are 28x28 pixels with 3 color channels)
# 
# 2. **Custom Layers**: 
#    - GlobalAveragePooling2D: Reduces each feature map to a single value.
#    - Dense Layer: 
#      - Units: 128
#      - Activation: ReLU
#    - Dropout Layer: 
#      - Rate: 0.5 (to prevent overfitting)
#    - Dense Layer: 
#      - Units: 4 (assuming 4 classes for classification)
#      - Activation: Softmax
# 
# The model is compiled with the following parameters:
# - **Optimizer**: Adam
# - **Loss Function**: Categorical Crossentropy (suitable for multi-class classification)
# - **Metrics**: Accuracy (to monitor the accuracy during training and evaluation)
# 
# Design Choices:
# - The use of ResNet50 as the base model leverages pre-trained weights to improve feature extraction.
# - GlobalAveragePooling2D is used to reduce the spatial dimensions of the feature maps.
# - ReLU activation is chosen for its effectiveness in introducing non-linearity.
# - Dropout is applied to prevent overfitting.
# - The final Dense layer uses Softmax activation to output a probability distribution over the classes.
# - Adam optimizer is selected for its efficiency and adaptive learning rate capabilities.
# - Categorical Crossentropy is used as the loss function to handle one-hot encoded labels.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Load the ResNet50 model with pre-trained weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(28, 28, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert grayscale images to 3 channels by repeating the single channel 3 times
train_images_resnet = np.repeat(train_images_cnn, 3, axis=-1)
val_images_resnet = np.repeat(val_images_cnn, 3, axis=-1)
flattened_test_images_resnet = np.repeat(flattened_test_images_cnn, 3, axis=-1)

# Train the model
model.fit(train_images_resnet, train_labels_cnn, epochs=10, batch_size=32, validation_data=(val_images_resnet, val_labels_cnn))

# Predict on test set
test_preds_resnet = model.predict(flattened_test_images_resnet)
test_preds_resnet_labels = np.argmax(test_preds_resnet, axis=1)

# Convert predictions to original array format
predictions_resnet = make_original_array(test_preds_resnet_labels)

# Create a DataFrame and save to CSV
df_resnet = pd.DataFrame(predictions_resnet, columns=['ID', 'label'])
df_resnet.to_csv('ResNet_baseline1_attempt.csv', index=False)


## ResNet50 attempt using PyTorch
Parameters: 100 epochs, learning rate=0.001, batch size=32, dropout rate=0.5

Preprocessing: Convert grayscale images to 3 channels by repeating the single channel 3 times

Model Architecture:
- Base Model: ResNet50 with pre-trained weights (ImageNet)
- Custom Layers:
    - Global Average Pooling
    - Dense Layer: 128 units, ReLU activation
    - Dropout Layer: 0.5 rate
    - Dense Layer: 4 units, Softmax activation

Training:
- Loss Function: CrossEntropyLoss
- Optimizer: Adam

Evaluation:
- Validation Accuracy and Loss per epoch
- Predictions on test set saved to CSV file

In [ ]:
import torchvision.models as models
import torch.optim as optim

# Load the ResNet50 model with pre-trained weights for retina medical images
base_model = models.resnet50(weights='IMAGENET1K_V2').to(device)
base_model.fc = nn.Identity()  # Remove the final fully connected layer

# Add custom layers on top of the base model
class CustomResNet(nn.Module):
    def __init__(self, base_model):
        super(CustomResNet, self).__init__()
        self.base_model = base_model
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(base_model.fc.in_features, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.base_model(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

model = CustomResNet(base_model).to(device)

# Freeze the layers of the base model
for param in base_model.parameters():
    param.requires_grad = False

# Convert grayscale images to 3 channels by repeating the single channel 3 times
train_images_resnet = np.repeat(train_images_cnn, 3, axis=1)
val_images_resnet = np.repeat(val_images_cnn, 3, axis=1)
flattened_test_images_resnet = np.repeat(flattened_test_images_cnn, 3, axis=1)

# Convert data to PyTorch tensors
train_images_tensor = torch.tensor(train_images_resnet, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
val_images_tensor = torch.tensor(val_images_resnet, dtype=torch.float32).to(device)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long).to(device)
test_images_tensor = torch.tensor(flattened_test_images_resnet, dtype=torch.float32).to(device)

# Create DataLoader
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = val_correct / len(val_dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss}, Accuracy: {val_accuracy}')

# Predict on test set
model.eval()
with torch.no_grad():
    test_outputs = model(test_images_tensor)
    _, test_preds_resnet_labels = torch.max(test_outputs, 1)

# Convert predictions to original array format
predictions_resnet = make_original_array(test_preds_resnet_labels.numpy())

# Create a DataFrame and save to CSV
df_resnet = pd.DataFrame(predictions_resnet, columns=['ID', 'label'])
df_resnet.to_csv('ResNet_baseline1_attempt.csv', index=False)